In [1]:
%load_ext autoreload
%autoreload 2
  
import fiona
    
from demosearch import FileCache
from demosearch.search import ftsearch
import utm
from collections import defaultdict    


from pathlib import Path
import metapack as mp
import geopandas as gpd
import pandas as pd
import numpy as np
from auto_tqdm import tqdm 
import appnope

from demosearch.util import run_mp


doc = mp.jupyter.open_source_package()
doc.set_sys_path()
import  pylib 

import logging
logging.basicConfig()

from pylib import lines_logger, points_logger
lines_logger.setLevel(logging.DEBUG)
points_logger.setLevel(logging.DEBUG)

pkg_root = Path(doc.path).parent
pkg = mp.open_package(pkg_root)
cache = pylib.open_cache(pkg)
pkg



# Open Street Map Extract for Demographic Search
`civicknowledge.com-osm-demosearch-2.1.1` Last Update: 2021-03-19T17:31:05

__




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [point_tags](data/point_tags.csv)**. Points converted to counts of tags per geohash
* ** [residential_roads](data/residential_roads.csv)**. Residential roads per 4 digit geohash
* ** [nonres_roads](data/nonres_roads.csv)**. Non residential roads per 4 digit geohash
* ** [block_geo](data/block_geo.csv)**. Geographic shapes for blocs in block_osm
* ** [block_osm](data/block_osm.csv)**. Points from geohash_tags allocated to 2020 census blocks
* ** [utm_map](data/utm_map.csv)**. Map from census blocks to UTM zones
* ** [cbsa_map](data/cbsa_map.csv)**. Map from census blocks to CBSA Metros
* ** [business_clusters](data/business_clusters.csv)**. High density business clusters in CBSAs
* ** [metro_points](data/metro_points.csv)**. Points grouped into major categories and linked to business clusters.
* ** [bus_densities](notebooks/PointDensity.ipynb#bus_densities)**. High-density business goehashes

## References
<ul><li> <strong>us_geohashes</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-geohash-us.csv#us_geohashes</em>. All 4 digit geohases in the continential US</li><li> <strong><a href="https://download.geofabrik.de/north-america-latest.osm.pbf">north-america-latest</a></strong>. OSM North America extract</li><li> <strong>utm_grid</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-mgrs.csv#utm_grid</em>. </li><li> <strong>points</string>, <em>data/csv/points.csv</em>. Points from the OSM file</li><li> <strong>lines</string>, <em>data/csv/lines.csv</em>. Lines from the OSM file</li><li> <strong>multipolygons</string>, <em>data/csv/multipolygons.csv</em>. Polygons from the OSM file</li><li> <strong>multilinestrings</string>, <em>data/csv/multilinestrings.csv</em>. Lines from the OSM file</li><li> <strong>other_relations</string>, <em>data/csv/other_relations.csv</em>. Other geo data from the OSM file</li><li> <strong>block_templ</string>, <em>censusgeo://2020/5/{st}/block</em>. Block url template</li><li> <strong>cbsa</string>, <em>censusgeo://2020/5/US/cbsa</em>. Metro areas</li><ul>

In [2]:
pkg

# Open Street Map Extract for Demographic Search
`civicknowledge.com-osm-demosearch-2.1.1` Last Update: 2021-03-19T17:31:05

__




 

## Contacts

* **Wrangler** [Eric Busboom](mailto:eric@civicknowledge.com), [Civic Knowledge](http://civicknowledge.com)

## Resources

* ** [point_tags](data/point_tags.csv)**. Points converted to counts of tags per geohash
* ** [residential_roads](data/residential_roads.csv)**. Residential roads per 4 digit geohash
* ** [nonres_roads](data/nonres_roads.csv)**. Non residential roads per 4 digit geohash
* ** [block_geo](data/block_geo.csv)**. Geographic shapes for blocs in block_osm
* ** [block_osm](data/block_osm.csv)**. Points from geohash_tags allocated to 2020 census blocks
* ** [utm_map](data/utm_map.csv)**. Map from census blocks to UTM zones
* ** [cbsa_map](data/cbsa_map.csv)**. Map from census blocks to CBSA Metros
* ** [business_clusters](data/business_clusters.csv)**. High density business clusters in CBSAs
* ** [metro_points](data/metro_points.csv)**. Points grouped into major categories and linked to business clusters.
* ** [bus_densities](notebooks/PointDensity.ipynb#bus_densities)**. High-density business goehashes

## References
<ul><li> <strong>us_geohashes</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-geohash-us.csv#us_geohashes</em>. All 4 digit geohases in the continential US</li><li> <strong><a href="https://download.geofabrik.de/north-america-latest.osm.pbf">north-america-latest</a></strong>. OSM North America extract</li><li> <strong>utm_grid</string>, <em>metapack+http://library.metatab.org/civicknowledge.com-mgrs.csv#utm_grid</em>. </li><li> <strong>points</string>, <em>data/csv/points.csv</em>. Points from the OSM file</li><li> <strong>lines</string>, <em>data/csv/lines.csv</em>. Lines from the OSM file</li><li> <strong>multipolygons</string>, <em>data/csv/multipolygons.csv</em>. Polygons from the OSM file</li><li> <strong>multilinestrings</string>, <em>data/csv/multilinestrings.csv</em>. Lines from the OSM file</li><li> <strong>other_relations</string>, <em>data/csv/other_relations.csv</em>. Other geo data from the OSM file</li><li> <strong>block_templ</string>, <em>censusgeo://2020/5/{st}/block</em>. Block url template</li><li> <strong>cbsa</string>, <em>censusgeo://2020/5/US/cbsa</em>. Metro areas</li><ul>

In [3]:
fc = FileCache('/Volumes/SSD_Extern/radius/')
def ftbb(s):
    bb = ftsearch(fc, s)[0].bb
    return (slice(bb[0],bb[2]), slice(bb[1],bb[3]))

ftbb('San Diego')
r=ftsearch(fc, 'San Diego')[0]

r

CBSAResult(lat=33.0236041, lon=-116.7761174, pt=(33.0236041, -116.7761174), area=817317026.0, bb=(-117.611081, 32.528832, -116.08094, 33.505024999999996), name='San Diego-Chula Vista-Carlsbad, CA', geoid='31000US41740')

In [4]:
from pylib.clusters import * 

def select_pb(df):
    return df.to_crs(4326).cx[-117.26:-117.22,32.78:32.82]

geoid = '31000US41740'
cache = get_cache(pkg)
